In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

: 

# Loading Data

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
iris  = load_iris()
X = iris.data
y = iris.target

In [ ]:
iris

In [ ]:
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df['species'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

In [ ]:
df.head(100)

# Preprocessing Data

In [ ]:
features = iris.feature_names
X = df[features].values
y = df['target'].values
X.shape

In [ ]:
y.shape

# Spliting Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"\nBefore scaling - Feature ranges:")
for i in range(X.shape[1]):
    print(f"Feature {i}: Min={np.min(X[:, i])}, Max={np.max(X[:, i])}")

print(f"\nAfter scaling - Feature ranges:")
for i in range(X_train.shape[1]):
    print(f"Feature {i}: Min={np.min(X_train[:, i])}, Max={np.max(X_train[:, i])}")

# One hot encoding

In [ ]:
y_train_cat = to_categorical(y_train,3)
y_test_cat = to_categorical(y_test,3)

print(f"\nOne-hot encoded labels shape: {y_train_cat.shape}")
print(f"Sample label before encoded: ({y_train[0]} ({iris.target_names[y_train[0]]}))")
print(f"Sample label after encoded: {y_train_cat[0]}")

In [ ]:
y_train_cat

# Neural Network model building

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(4,), name='input_layer'),
    layers.Dense(64, activation='relu', name = 'hidden_layer_1'),
    layers.Dropout(0.3,name='dropout_1'),
    layers.Dense(32, activation='relu', name = 'hidden_layer_2'),
    layers.Dropout(0.3,name='dropout_2'),
    layers.Dense(16, activation='relu', name = 'hidden_layer_3'),
    layers.Dense(3, activation='softmax', name = 'output_layer')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Training

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
history = model.fit(X_train, y_train_cat, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Model Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# Visualization

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(1, 4, 1)
plt.plot(history.history['loss'], label='Training Loss',linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss',linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True,)

#Model Perfomance

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 4, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy',linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy',linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)

In [ ]:
print(f"Final Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Final Test Loss: {test_loss:.4f}")
print(f"Total trainable parameters: {model.count_params():,}")

# Per-class accuracy
print(f"\nPer-class performance:")
for i, species in enumerate(iris.target_names):
    class_mask = (y_test == i)
    if sum(class_mask) > 0:
        class_acc = np.mean(y_pred[class_mask] == y_test[class_mask])
        print(f"{species}: {class_acc:.4f} ({class_acc*100:.1f}%)")

# Model complexity analysis
print(f"\nModel complexity:")
print(f"- Number of layers: {len(model.layers)}")
print(f"- Hidden layers: {len(model.layers) - 1}")
print(f"- Total parameters: {model.count_params()}")

# Training efficiency
print(f"\nTraining efficiency:")
print(f"- Epochs trained: {len(history.history['loss'])}")
print(f"- Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"- Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")


# Saving the Model

In [ ]:
model.save('my_model.keras')